In [1]:
import numpy as np

def generate_spatial_vector(dim):
    """Generate a random spatial vector of the specified dimension."""
    return np.random.randn(dim)

def generate_lorentz_vector(spatial_vector, kappa):
    """Generate a Lorentz vector with a specified spatial part and curvature."""
    # Ensure the argument inside the sqrt is positive
    spatial_norm_squared = np.sum(spatial_vector**2)
    if spatial_norm_squared - 1/abs(kappa) < 0:
        raise ValueError("Invalid vector components for the given curvature, leading to negative square root domain.")
    time_component = np.sqrt(spatial_norm_squared - 1/kappa)
    return np.concatenate([[time_component], spatial_vector])

def lorentz_inner_product(v1, v2):
    """Compute the Lorentz inner product given two Lorentz vectors."""
    return -v1[0] * v2[0] + np.dot(v1[1:], v2[1:])

def lorentz_distance(v1, v2, kappa):
    """Compute the Lorentzian distance based on the Lorentz inner product."""
    inner_product = kappa * lorentz_inner_product(v1, v2)
    if inner_product < 1:
        return np.nan  # Adjusted inner product is below the domain of arcosh
    return np.sqrt(abs(1/kappa)) * np.arccosh(inner_product)

def check_distance_preservation(num_samples, dimension, kappa1, kappa2):
    fail_count = 0
    for _ in range(num_samples):
        try:
            # Generate spatial components for vectors x, y, z
            x_space = generate_spatial_vector(dimension - 1)
            y_space = generate_spatial_vector(dimension - 1)
            z_space = generate_spatial_vector(dimension - 1)

            # Generate Lorentz vectors for both curvatures
            x_lorentz_kappa1 = generate_lorentz_vector(x_space, kappa1)
            y_lorentz_kappa1 = generate_lorentz_vector(y_space, kappa1)
            z_lorentz_kappa1 = generate_lorentz_vector(z_space, kappa1)
            x_lorentz_kappa2 = generate_lorentz_vector(x_space, kappa2)
            y_lorentz_kappa2 = generate_lorentz_vector(y_space, kappa2)
            z_lorentz_kappa2 = generate_lorentz_vector(z_space, kappa2)

            # Compute distances for both curvatures
            distance_xy_1 = lorentz_distance(x_lorentz_kappa1, y_lorentz_kappa1, kappa1)
            distance_xz_1 = lorentz_distance(x_lorentz_kappa1, z_lorentz_kappa1, kappa1)
            distance_xy_2 = lorentz_distance(x_lorentz_kappa2, y_lorentz_kappa2, kappa2)
            distance_xz_2 = lorentz_distance(x_lorentz_kappa2, z_lorentz_kappa2, kappa2)

            # Check if the relative order is preserved
            if (distance_xz_1 > distance_xy_1) != (distance_xz_2 > distance_xy_2):
                fail_count += 1

        except ValueError as e:
            continue  # Skip current iteration and continue with the next sample

    return fail_count

# Example parameters
num_samples = 50000
dimension = 4
kappa1 = -1
kappa2 = -5

# Running the test
fail_count = check_distance_preservation(num_samples, dimension, kappa1, kappa2)
print("Number of fail cases:", fail_count)


Number of fail cases: 174


To complete the proof, we will analyze the impact of changing curvature on the Lorentzian distances and determine whether the relative ordering of distances is preserved under this transformation.

### Mathematical Setup:

We have vectors $\mathbf{x}$, $\mathbf{y}$, and $\mathbf{z}$ in a Lorentzian space with curvatures $\kappa_1$ and $\kappa_2$. For each vector $\mathbf{a}$, the Lorentz transformation due to a change in curvature results in new temporal components:
$$ \mathbf{a} = \left(\sqrt{\| \mathbf{\bar{a}} \|_2^2 - \frac{1}{\kappa_1}}, \mathbf{\bar{a}}\right), $$
$$ \mathbf{a}' = \left(\sqrt{\| \mathbf{\bar{a}} \|_2^2 - \frac{1}{\kappa_2}}, \mathbf{\bar{a}}\right). $$

### Lorentz Inner Product and Distance Calculation:

The Lorentz inner product is defined as:
$$ \langle \mathbf{a}, \mathbf{b} \rangle_\mathcal{L} = -a_0b_0 + \mathbf{\bar{a}}^T\mathbf{\bar{b}}, $$
where $a_0$ and $b_0$ are the temporal components of vectors $\mathbf{a}$ and $\mathbf{b}$, respectively.

The Lorentzian distance between two points $\mathbf{a}$ and $\mathbf{b}$ under curvature $\kappa$ is:
$$ d_{\mathcal{L}}^\kappa(\mathbf{a}, \mathbf{b}) = \sqrt{\frac{1}{|\kappa|}}\text{arcosh}\left(\kappa \langle \mathbf{a}, \mathbf{b} \rangle_\mathcal{L}\right). $$

### Effect of Curvature Change on Distance:

Upon changing the curvature from $\kappa_1$ to $\kappa_2$, the temporal components of the vectors change, affecting the Lorentz inner product:
$$ \langle \mathbf{a}', \mathbf{b}' \rangle_{\mathcal{L}} = -a_0' b_0' + \mathbf{\bar{a}}^T \mathbf{\bar{b}}, $$
where:
$$ a_0' = \sqrt{\|\mathbf{\bar{a}}\|_2^2 - \frac{1}{\kappa_2}}, \quad b_0' = \sqrt{\|\mathbf{\bar{b}}\|_2^2 - \frac{1}{\kappa_2}}. $$

The change in curvature affects the magnitude of the temporal components, which in turn affects the first term of the inner product, $-a_0' b_0'$. Since the curvature $\kappa$ scales this term, the impact on the Lorentzian distance involves both the magnitude of this term and the sign of $\kappa$.

### Preservation of Relative Distance Order:

Assume we initially have $d_{\mathcal{L}}^{\kappa_1}(\mathbf{x}, \mathbf{z}) > d_{\mathcal{L}}^{\kappa_1}(\mathbf{x}, \mathbf{y})$, translating to:
$$ \text{arcosh}\left(\kappa_1 (-x_0z_0 + \mathbf{\bar{x}}^T \mathbf{\bar{z}})\right) > \text{arcosh}\left(\kappa_1 (-x_0y_0 + \mathbf{\bar{x}}^T \mathbf{\bar{y}})\right). $$

When curvature changes, we analyze:
$$ \text{arcosh}\left(\kappa_2 (-x_0'z_0' + \mathbf{\bar{x}}^T \mathbf{\bar{z}})\right) \quad \text{vs} \quad \text{arcosh}\left(\kappa_2 (-x_0'y_0' + \mathbf{\bar{x}}^T \mathbf{\bar{y}})\right). $$

The critical insight is that even though the spatial components remain constant, the change in the sign and magnitude of the temporal product term due to the curvature change can affect the order of these arcosh values. Since the arcosh function is monotonically increasing, any decrease in the Lorentz inner product value (made less negative or more positive) can potentially reverse the inequality.

### Conclusion:

Changing curvature can indeed alter the relative distances due to the non-linear dependence on the temporal components in the Lorentz inner product. The preservation of relative distance order is not guaranteed unless additional constraints on the relationship between $\mathbf{\bar{x}}$, $\mathbf{\bar{y}}$, and $\mathbf{\bar{z}}$ or specific properties of $\kappa_1$ and $\kappa_2$ are imposed. This sensitivity to curvature demonstrates the complex geometry of Lorentz spaces and has implications in physical theories like general relativity, where curvature plays a fundamental role.